In [1]:
from pathlib import Path
import sqlite3
from typing import List

import pandas as pd

In [2]:
!wget https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true -O "rpg_db.sqlite3"

--2020-08-12 13:57:28--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2020-08-12 13:57:28--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2020-08-12 13:57:28--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-in

In [3]:
_valid_contexts: List[str] = ['sqlite',
                              # 'postgres',
                              # 'mongo'
                              ]

In [4]:
def _valid_db_path(pattern: str):
    """Raise an exception in the event of invalid database."""
    # Let's check first that we've been passed an object with type
    # of string
    if not isinstance(pattern, str):
        raise TypeError(f'{pattern} invalid type for database path.')
    # If this check passes then we could set a variable.from the pattern
    # argument. Using ``Path().absolute()`` we can retrieve the absolute
    # path (eg. entire path starting from the root [C:// or /home/)])
    # to the supposed database.
    path = Path(pattern).absolute()
    # We know that the sqlite3 module will create a database if it
    # does not already exist, so perhaps it would be more helpful
    # to ensure that the path does not point to a directory which would
    # be invalid.
    if path.is_dir():
        raise ValueError(f"'{path}' is a directory not an sqlite database.")
    # Finally if both the above checks pass it may be more helpful to
    # pass back to the class the absolute path.
    return str(path)

In [40]:
_valid_db_path('/content/')

ValueError: ignored

In [25]:
class DbHelper(object):
    """Helper class for interacting with SQL databases."""

    def __init__(self, path_to_database: str, context: str) -> None:
        f"""
        :param path_to_database: Database path.
        :param context: The type of database to use one of {str(" ".join(_valid_contexts))}
        """
        # First we should validate that the context is a valid selection
        # if it is not we should raise an error informing the user.
        if context not in _valid_contexts:
            # Prettify the list of valid contexts
            contexts = " ".join(_valid_contexts)
            raise ValueError(f"'{context}' is not a valid context.",
                             '\n',
                             f"You should try one of these: {contexts}"
                             )
        # If a valid context has been selected then lets set variable
        self._context = context

        # Next we should use some sort of validation to ensure that the
        # value being passed as the database location is indeed a valid
        # selection. This is what the above function was defined for,
        # so we can set the instance attribute of the database location
        # to the result of the function
        self._db_location = _valid_db_path(path_to_database)
        # We can establish a connection upon instantiation by calling
        # the ``_connect`` method here
        self.connection, self.cursor = self._connect()

    def _connect(self):
        """Establish a connection to the database."""
        # This is fairly straightforward as we're merely using the
        # code that we're already familiar with it and executing it
        # inside of a class method.
        #
        # We'll use the naming convention to tell other programmers that
        # this is a private method.
        #
        # We can also begin to define the logic which will allow us to
        # later extend this class to work with other contexts
        if self._context == 'sqlite':
            connection = sqlite3.connect(self._db_location)
            cursor = connection.cursor()
            return connection, cursor

    def execute(self, statement: str):
        """Execute a single query."""
        # Again we're merely defining the logic we're already familiar
        # with inside of the class method.
        self.connection.execute(statement)


    def query(self, query: str):
        """Execute a query."""
        # A query is just an executed statement that returns
        # data.
        self.cursor.execute(query)
        data = self.cursor.fetchall()
        return data

    def commit(self):
        """Commit changes to the database."""
        self.connection.commit()

    def close(self):
        """Close the connection."""
        self.connection.close()

    def __enter__(self):
        # This just returns the instance itself
        return self

    def __exit__(self, ext_type, exc_value, traceback):
        # First close the cursor
        self.cursor.close()
        # If an exception has been encountered then
        if isinstance(exc_value, Exception):
            # rollback the database
            self.connection.rollback()
        # otherwise commit the changes to the database
        else:
            self.commit()
        # close the connection
        self.close()


In [36]:
db = DbHelper(path_to_database='/content/rpg_db.sqlite3', context='sqlite')

In [39]:
db.query(query="""
SELECT COUNT (DISTINCT name) AS Number_of_total_Characters
FROM charactercreator_character
""")

[(297,)]

In [34]:
# Having defined both __enter__ and __exit__ methods
# we can utilize context management and execute
# our query inside of a ``with`` statement to ensure
# the connection is properly closed upon exiting the
# block of code.

with DbHelper(
    path_to_database='/content/rpg_db.sqlite3', context='sqlite'
    ) as db:
    answer = db.query(
"""
SELECT COUNT (DISTINCT name) AS Number_of_total_Characters
FROM charactercreator_character
"""
)
    
print(answer)

[(297,)]
